In [ ]:
#
%matplotlib inline
import matplotlib
import matplotlib_inline.backend_inline
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg", "pdf", "retina")  # For export
import sys 
sys.path.append("../src/")
import seaborn as sns 
import altair as alt
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from cycler import cycler
az.style.use(["science", 'arviz-doc'])
nice_fonts = {
        # Use LaTeX to write all text
        "font.family": "serif",
        # Always save as 'tight'
        "savefig.bbox" : "tight",
        "savefig.pad_inches" : 0.05,
        "ytick.right" : True,
        "font.serif" : "Times New Roman",
        "mathtext.fontset" : "dejavuserif",
        "axes.labelsize": 15,
        "font.size": 15,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": 10,
        'legend.facecolor':'white',
        'legend.edgecolor': 'gray',
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
        # Set line widths
        "axes.linewidth" : 0.5,
        "grid.linewidth" : 0.5,
        "lines.linewidth" : 1.,
        # Remove legend frame
        "legend.frameon" :False,
        # color-blind friendly cycle designed using https://colorcyclepicker.mpetroff.net/
        # see preview and check for colorblindness here https://coolors.co/107591-00c0bf-f69a48-fdcd49-8da798-a19368-525252-a6761d-7035b7-cf166e
        'axes.prop_cycle': cycler(color=['#107591','#00c0bf','#f69a48','#fdcd49','#8da798','#a19368','#525252','#a6761d','#7035b7','#cf166e'])      
}
matplotlib.rcParams.update(nice_fonts)
from utils.data_processing import add_time_features
import altair as alt
alt.themes.enable("opaque")
alt.data_transformers.disable_max_rows()


## Define metrics functions

In [ ]:
## Metrics functions
from sklearn.metrics import r2_score
from utils.significant_test import run_statistical_test
from scipy.stats import norm, probplot,  anderson, shapiro
def _divide_no_nan(a: float, b: float) -> float:
    div = a / b
    div[div != div] = 0.0
    div[div == float('inf')] = 0.0
    return div

def get_mae(y, y_hat):
    return np.abs(y - y_hat)

def get_mse(y, y_hat):
    return np.square(y - y_hat)

def get_mse(y, y_hat):
    return np.square(y - y_hat)

def get_rmse(y, y_hat):
    return np.sqrt(get_mse(y, y_hat))


def get_mape(y, y_hat):
    delta_y = get_mae(y, y_hat)
    scale = np.abs(y)
    mape=_divide_no_nan(delta_y, scale)
    return mape

def get_smape(y, y_hat):
    delta_y = get_mae(y, y_hat)
    scale = np.abs(y) + np.abs(y_hat)
    smape = 2 *_divide_no_nan(delta_y, scale)
    return smape

def get_bias(y, y_hat):
    bias=(y - y_hat)/get_mae(y, y_hat)
    return bias

def get_residual(y, y_hat):
    res=(y - y_hat)
    return res

def get_nbias(y, y_hat):
    nbias=(y-y_hat)/(y + y_hat)
    return nbias

def get_pointwise_metrics(pred:np.array, true:np.array, pred_naive:np.array, true_naive):
    """calculate pointwise metrics
    Args:   pred: predicted values
            true: true values
            target_range: target range          
    Returns:    rmse: root mean square error                


    """
    true, pred = true.flatten(), pred.flatten()
    pred_naive, true_naive = pred_naive.flatten(), true_naive.flatten()
    bias=get_bias(true, pred).sum()
    nbias=get_nbias(true, pred).sum()
    mae=get_mae(true, pred).mean()
    mape=get_mape(true, pred).mean()
    smape=get_smape(true, pred).mean()
    rmse=np.sqrt(get_mse(true, pred).mean())
    rmae=get_mae(true, pred).mean()/get_mae(true_naive, pred_naive).mean()
    r2=r2_score(pred, true)
    res=get_residual(true, pred).mean()
    return dict(rmse=rmse, mae=mae, r2=r2, mape=mape, smape=smape,
                   bias=bias, nbias=nbias, rmae=rmae, res=res)

def get_daily_metrics(pred:np.array, true:np.array, pred_naive:np.array, true_naive):
    metrics=[]
    for i in range(len(true)):
        metric = get_pointwise_metrics(pred[i], true[i], pred_naive[i], true_naive[i])
        metrics.append(pd.DataFrame.from_dict(metric, orient='index').T)
    return pd.concat(metrics)


# functions to get all metrics
def get_results_per_experiment(models, dataset, experiment_name, window_type, cross_valid=list(range(1,11))):
    file_name=f'{experiment_name}_{window_type}'
    all_metrics=[]
    all_residual={}
    all_data=[]
    # [ 'TFT', 'NBEATS',  'NHiTS', 'RNN', 'MSTL', 'SeasonalNaive', 'AutoARIMA',   "RF", 'CATBOOST', 'LREGRESS']
    for encoder_type in models:
        metrics_per_model=[]
        residual_per_model=[]
        data_per_model=[]
        print(encoder_type)
        print()
        for cross in tqdm(cross_valid):
            if window_type=='train_test':
                result_path=f'../results/{file_name}/{encoder_type}/{cross}_processed_results.npy'
                result_path_naive=f'../results/{file_name}/SeasonalNaive/{cross}_processed_results.npy'
            else:
                result_path=f'../results/{file_name}/{encoder_type}/{cross}_cross_validation_processed_results.npy'
                result_path_naive=f'../results/{file_name}/SeasonalNaive/{cross}_cross_validation_processed_results.npy'
            results=np.load(result_path, allow_pickle=True).item()
            results_naive=np.load(result_path_naive, allow_pickle=True).item()
            time_stamp=results['index']
            df = pd.DataFrame(time_stamp.flatten())
            df.columns=['Date']
            index=df.Date.dt.round("D").unique()
            metrics=results['NetLoad_metrics']
            true=results['true']
            pred=results['pred']
            
            
            df=pd.DataFrame(index=results['index'].flatten())
            df[f'{encoder_type}']=pred.flatten()
            df['true']=true.flatten()
            data_per_model.append(df)
            

    

            residual=true-pred
            residual_per_model.append(residual.flatten())
            pred_naive=results_naive['pred']
            true_naive=results_naive['true']
            metrics=get_daily_metrics(pred, true, pred_naive, true_naive)
            metrics['timestamp']=index[:len(metrics)]
            metrics['Fold']=cross
            metrics['Model']=encoder_type
            metrics['Train-time']=results['train-time'] if 'train-time' in results.keys() else results['train_time']
            metrics['Test-time']=results['test-time'] if 'test-time' in results.keys() else results['test_time']
            metrics=metrics.set_index('timestamp')
            metrics=add_time_features(metrics, hemisphere = 'Northern')
            metrics=metrics.replace('CATBOOST', 'CAT').replace('AutoARIMA', 'ARIMA').replace('LREGRESS', 'L-REG').replace('SeasonalNaive', 'S-Naive').replace('D-LINEAR', 'D-LIN')
            metrics_per_model.append(metrics)
        all_residual[encoder_type]=residual_per_model
        metrics_per_model=pd.concat(metrics_per_model)
        all_metrics.append(metrics_per_model)
        all_data.append(pd.concat(data_per_model))

    metrics=pd.concat(all_metrics)
    if dataset=="UK":
        metrics['Dataset']='SPS-UK}T'
    else:
        metrics['Dataset']='MLVS-PT'
    all_data=pd.concat(all_data, axis=1)
    del all_metrics
    del df 
    del results
    del results_naive
    del pred
    del true
    
    return metrics, all_residual, all_data

In [ ]:
forecast_len=6 
incremental_len = 2
min_train_len=12
seed=777
max_epochs=50                   
window_type='expanding'          
dataset="PT",
experiment_name="PT-BestPractises"

## Experiment 1: Metrics

#### Objectives: 

- To un-
derstand their suitability in accurately assessing net
load forecasting, emphasizing critical aspects such  s
peak values and negative net-l and how each metric rank the baselines models
- Metrics considered: NRMSE, MAE, SMAPE, MAPE, NBIAS

ad.

In [ ]:
models=['MSTL', 'SeasonalNaive',   "RF",  'LREGRESS', 'CATBOOST'] #,

def get_results_per_dataset(models, dataset, experiment_name, window_type):
    
    file_name=f'{experiment_name}_{window_type}'
    all_metrics=[]
    all_residual={}
    for encoder_type in models:
        print(encoder_type)
        print()
        for cross in tqdm(range(1, 11)):
            result_path=f'../results/{file_name}/{encoder_type}/{cross}_cross_validation_processed_results.npy'
            results=np.load(result_path, allow_pickle=True).item()
            time_stamp=results['index'].flatten()
    
            metrics=results['NetLoad_metrics']
            df=pd.DataFrame(index=time_stamp)
            df['pred']=results['pred'].flatten()
            df['true']=results['true'].flatten()
            all_metrics.append(df)
            
    results = pd.concat(all_metrics)
    if dataset=="UK":
        results['Dataset']='SPS-UK}T'
    else:
        results['Dataset']='MLVS-PT'
    results['mae']=get_mae(results['true'].values, results['pred'].values)
    results['bias']=get_bias(results['true'].values, results['pred'].values)
    results['nbias']=get_nbias(results['true'].values, results['pred'].values)
    results['mse']=get_mse(results['true'].values, results['pred'].values)
    results['rmse']=results['mse']**0.5
    results['mape']=get_mape(results['true'].values, results['pred'].values)
    results['residual']=results['true'].values-results['pred'].values
    results['smape']=get_smape(results['true'].values, results['pred'].values)
    
    results['timestamp']=results.index
    results=results.set_index('timestamp')
    results=add_time_features(results, hemisphere = 'Northern')
    
    return results
    

In [ ]:
results=get_results_per_dataset(['CATBOOST'], dataset, experiment_name, window_type)
lowest_netload=np.where(results['true']<0)[0]
highest_netload=np.where(results['true']>90)[0]

### Metrics with peaks and valley PT'dataset

In [ ]:
figure_path='../figures/SmartGridPaper/'

In [ ]:
df.columns

In [ ]:
## Metrics at peak and valley
idx=highest_netload[2]
fig, ax = plt.subplots(2, 3, figsize=(10, 4.0), sharex=False)
ax = ax.ravel()
df=results.iloc[idx-48:idx+48]
df['nrmse']=df.rmse/df.true.max()
#df=df/df.max(axis=0)
ax[0].plot(df['true'].values, ".", mec="C2", mfc="None", label='True')
ax[0].plot(df['pred'].values,  '--',  c='C0', label='Pred')
ax[0].set_ylabel('Power (kW)')
ax[0].legend(loc='upper left', fontsize=10)

ax2 = ax[0].twinx()
ax2.plot(df['mae'].values, c='C1', label='MAE')
ax2.set_ylabel('MAE')
ax2.legend(loc='upper right', fontsize=10)

ax[1].plot(df['true'].values, ".", mec="C2", mfc="None", label='True')
ax[1].plot(df['pred'].values,  '--',  c='C0', label='Pred')
ax[1].set_ylabel('Power (kW)')
#ax[1].legend(loc='upper left')
ax3 = ax[1].twinx()
ax3.plot(df['mape'].values, c='C1', label='MAPE')
ax3.set_ylabel('MAPE')
ax3.legend(loc='upper right', fontsize=10)


ax[2].plot(df['true'].values, ".", mec="C2", mfc="None", label='True')
ax[2].plot(df['pred'].values,  '--',  c='C0', label='Pred')
ax[2].set_ylabel('Power (kW)')
#ax[2].legend(loc='upper left')
ax3 = ax[2].twinx()
ax3.plot(df['smape'].values, c='C1', label='SMAPE')
ax3.set_ylabel('SMAPE')
ax3.legend(loc='upper right', fontsize=10)


ax[3].plot(df['true'].values, ".", mec="C2", mfc="None", label='True')
ax[3].plot(df['pred'].values,  '--',  c='C0', label='Pred')
ax[3].set_ylabel('Power (kW)')
#ax[3].legend(loc='upper left')
ax3 = ax[3].twinx()
ax3.plot(df['nrmse'].values, c='C1', label='NRMSE')
ax3.set_ylabel('NRMSE')
ax3.legend(loc='upper right', fontsize=10)

ax[4].plot(df['true'].values, ".", mec="C2", mfc="None", label='True')
ax[4].plot(df['pred'].values,  '--',  c='C0', label='Pred')
ax[4].set_ylabel('Power (kW)')
#ax[3].legend(loc='upper left')
ax3 = ax[4].twinx()
ax3.plot(df['nbias'].values, c='C1', label='NBIAS')
ax3.set_ylabel('NBIAS')
ax3.legend(loc='upper right', fontsize=10)

fig.tight_layout()
#fig.savefig(f'{figure_path}/metrics_with_valley_and_peaks.pdf', dpi=480)

In [ ]:
fig=sns.jointplot(data=df, x="residual", y="mae", kind="reg", palette="tab10", height=2.8, color='C6')
fig.set_axis_labels('residual', 'MAE', fontsize=12)
fig.figure.tight_layout()
#fig.savefig(f'{figure_path}/metrics_with_valley_and_peaks_joint_mae.pdf', dpi=480)

In [ ]:
fig=sns.jointplot(data=df, x="residual", y="mape", kind="reg", palette="tab10", height=2.8, color='C6')
fig.set_axis_labels('residual', 'MAPE', fontsize=12)
fig.figure.tight_layout()
#fig.savefig(f'{figure_path}/metrics_with_valley_and_peaks_joint_mape.pdf', dpi=480)

In [ ]:
fig=sns.jointplot(data=df, x="residual", y="smape", kind="reg", palette="tab10", height=2.8, color='C6')
fig.set_axis_labels('residual', 'SMAPE', fontsize=12)
fig.figure.tight_layout()
#fig.savefig(f'{figure_path}/metrics_with_valley_and_peaks_joint_smape.pdf', dpi=480)

In [ ]:
fig=sns.jointplot(data=df, x="residual", y="nrmse", kind="reg", palette="tab10", height=2.8, color='C6')
fig.set_axis_labels('residual', 'NRMSE', fontsize=12)
fig.figure.tight_layout()
#fig.savefig(f'{figure_path}/metrics_with_valley_and_peaks_joint_mse.pdf', dpi=480)

In [ ]:
fig=sns.jointplot(data=df, x="residual", y="nbias", palette="tab10", height=2.8,  color='C6')
fig.set_axis_labels('residual', 'NBIAS', fontsize=12)
fig.figure.tight_layout()
#fig.savefig(f'{figure_path}/metrics_with_valley_and_peaks_joint_nbias.pdf', dpi=480)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.,2.0))
sns.histplot(results.residual.values, kde=True, ax=ax, color='C6')
ax.axvline(results.residual.values.mean(), color='r', linestyle="--")
ax.set_xlabel("Residual", fontsize=10)
ax.set_ylabel("Counts", fontsize=10)
fig.tight_layout()
#fig.savefig(f'{figure_path}/residual_sample_.pdf', dpi=480)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.,2.1)) 
probplot(df.residual.values, dist="norm", plot=ax)
ax.set_xlabel("Theoretical Quantiles", fontsize=10)
ax.set_ylabel("Sample Quantiles", fontsize=10)
ax.set_title("", fontsize=10)
fig.tight_layout()
#fig.savefig(f'{figure_path}/QQplot_sample_.pdf', dpi=480)

In [ ]:
#fig, (ax1,ax2) = plt.subplots(1,2, figsize=(6,3)) 
#sns.scatterplot(data=results, x='residual', y='nbias', ax=ax1, hue='Season').set(title='Data')
#ax1.get_legend().remove()
#sns.move_legend(ax2, "upper left", bbox_to_anchor=(1.05, 0.8));

## Metrics across different models

In [ ]:
transformer_kva=250
p_factor=0.8
installed_capacity=results['true'].max()

In [ ]:
all_metrics={}
all_residual={}
all_data={}
for  window_type in ['expanding',  'train_test']:
    
    if window_type=='train_test':
        cross_valid=['test-two', 'test-one']
    else:
        cross_valid=list(range(1, 11))
    metrics, residual, data=get_results_per_experiment(models, dataset, experiment_name, window_type, cross_valid)
    metrics['window']=window_type
    all_metrics[window_type]=metrics
    all_residual[window_type]=residual
    all_data[window_type]=data
    
metrics=pd.concat(all_metrics.values())
metrics['nrmse']=metrics['rmse']/installed_capacity

In [ ]:
met=['nrmse', 'mae', 'mape', 'smape', 'nbias']
res=metrics[metrics['window']=='expanding'].groupby(['Model'])[met].mean().sort_values('mae').round(3)
#res=metrics[metrics['window']=='expanding'].groupby('Model')[met].std().sort_values('nrmse').round(2)
table = res.to_latex(index=True, formatters={"name": str.upper},
                  float_format="{:.3f}".format)
# Print the LaTeX table
res

### Analyse how each metric rank different models across seasons

In [ ]:
### Analyse how each metric rank different models across seasons

In [ ]:
metrics[metrics['window']=='expanding'].groupby([ 'Season','Model'])[met].mean()

In [ ]:
az.style.use(["science", 'arviz-doc'])
def per_metric_plot(data, metric='nrmse'):
    fig=sns.catplot(data=data, 
                    x='Model', 
                    y=metric, 
                    estimator='median',
                    hue='Season', 
                    hue_order=['Autumn', 'Winter', 'Spring', 'Summer'],
                    sharex=False, margin_titles=True,
                    height=3.25, aspect=1.0,
                    #order=['CAT',  'LSTM',  'NHiTS', 'NBEATS', "MLPFPQ"],
                    #palette={"MLVS-PT": "C1", "SPS-UK": "C2"},
                    order=[ 'CAT', 'L-REG',  'MSTL', 'S-Naive'],
                     kind='point',
                    legend='auto', legend_out=False)
    return fig
fig= per_metric_plot(metrics[metrics['window']=='expanding'].reset_index(), metric='mae')
fig.set_axis_labels( 'Model', 'MAE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper left', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(1, 15))
#fig.savefig(f'{figure_path}/Expanding_MAE.pdf', dpi=480)

In [ ]:
fig= per_metric_plot(metrics[metrics['window']=='expanding'].reset_index(), metric='mape')
fig.set_axis_labels( 'Model', 'MAPE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper left', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 0.4))
#fig.savefig(f'{figure_path}/Expanding_MAPE.pdf', dpi=480)

In [ ]:
fig= per_metric_plot(metrics[metrics['window']=='expanding'].reset_index(), metric='smape')
fig.set_axis_labels( 'Model', 'SMAPE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper left', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 0.4))
#fig.savefig(f'{figure_path}/Expanding_SMAPE.pdf', dpi=480)

In [ ]:
fig= per_metric_plot(metrics[metrics['window']=='expanding'].reset_index(), metric='nbias')
fig.set_axis_labels( 'Model', 'NBIAS', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='lower left', fontsize=10)
plt.xticks(rotation=90);

plt.axhline(y=2, color='black', linestyle='--', alpha=0.21)
plt.axhline(y=-2, color='black', linestyle='--', alpha=0.21)
fig.set(ylim=(-7.5, 8.0))
#fig.savefig(f'{figure_path}/Expanding_NBIAS.pdf', dpi=480)

In [ ]:
fig= per_metric_plot(metrics[metrics['window']=='expanding'].reset_index(), metric='nrmse')
fig.set_axis_labels( 'Model', 'NRMSE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper left', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 0.2))
#fig.savefig(f'{figure_path}/Expanding_NRMSE.pdf', dpi=480)

## Metrics agreebleness

In [ ]:
def get_correlation(data, variable='NetLoad', column=None):
    non_zero_varlist=[variable]+column
    correlations =  data[non_zero_varlist].corr().unstack().sort_values(ascending=False) # Build correlation matrix
    correlations = pd.DataFrame(correlations).reset_index() # Convert to dataframe
    correlations.columns = ['col1', 'col2', 'correlation'] # Label it
    load_corr=correlations.query(f"col1 == {variable} & col2 != {variable}")
    load_corr[np.abs(load_corr['correlation'])>=0.3]
    return load_corr[np.abs(load_corr['correlation'])>=0.3]

def plot_correlation(ax, data, variable=["NetLoad", "Load"], column=None):
    corr = data[variable+column].corr()
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    # Colors
    cmap = sns.diverging_palette(240, 10, as_cmap=True)
    ax=sns.heatmap(corr, mask=mask, linewidths=.5, cmap=cmap, center=0, annot=True, fmt=".1g")
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, horizontalalignment="right")
    ax.set_yticklabels(ax.get_yticklabels(), rotation=0, horizontalalignment="right")
    ax.set_title("Correlation Heatmap")
    return ax

In [ ]:
data=metrics[metrics['window']=='expanding']
data_rnn=data[data['Model'].isin(['RNN'])]
col_met=['nrmse', 'mae',  'mape', 'smape',  'nbias', 'res']
col=['NRME', 'MAE',  'MAPE', 'SMAPE',  'NBIAS', 'RES']
corr = data[col_met].corr()
cmap = sns.light_palette('#525252', as_cmap=True)
#cmap = sns.diverging_palette(240, 10, as_cmap=True)
fig, ax = plt.subplots(1,1, figsize=(5.5,5.5))
#plt.imshow(corr.astype(float).values, cmap=cmap)
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
ax=sns.heatmap(corr.astype(float).values, mask=mask, linewidths=.5, cmap=cmap, center=0, annot=True, fmt=".1g")
plt.xticks(range(len(col)), col, rotation=90.)
plt.yticks(range(len(col)), col)
ax.plot(range(corr.shape[0]), range(corr.shape[0]), 'wx')
#plt.colorbar(ax=ax)

## Statistical test

In [ ]:
data=all_data['expanding']
def get_net_load(data):
    net_load=data.true.iloc[:, 0]
    data=data[[  'MSTL','SeasonalNaive',  'RF',  'CATBOOST', 'LREGRESS']]
    data=data.rename(columns={'SeasonalNaive':'S-NAIVE',  'LREGRESS':'L-REG', 'CATBOOST':'CAT',  'AutoARIMA':'ARIMA'})
    data['NetLoad']=net_load.values
    # Transforming indices to datetime format
    data.index = pd.to_datetime(data.index)
    data.head()
    real_netload = data.loc[:, ['NetLoad']]
    del data['NetLoad']
    return data, real_netload

In [ ]:
data, real_netload=get_net_load(data)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_value=run_statistical_test(ax, real_netload, data, norm=2, test='GW')
#ax.set_title('')
#fig.savefig(f'{figure_path}/GW_test_model.pdf', dpi=480)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_values=p_value=run_statistical_test(ax, real_netload, data, norm=2, test='DM')
#ax.set_title('')
#fig.savefig(f'{figure_path}/DM_test_model.pdf', dpi=480)

Note: P-value close to zero represent cases where the forecast in the x-axis is significant more accurate than the forecast in the y-axis

In [ ]:
data=all_data['train_test']
data, real_netload=get_net_load(data)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(3.5,3))
p_values=p_value=run_statistical_test(ax, real_netload, data, norm=1, test='DM')
#fig.savefig(f'{figure_path}/DM_test_model_test_2.pdf', dpi=480)

## Metrics with different data spiliting

In [ ]:
def per_dtype_plot(data, metric='nrmse'):
    fig=sns.catplot(data=data, 
                    x='Model', 
                    y=metric, 
                    hue='D-type', 
                    estimator='median',
                    #hue_order=['Autumn', 'Winter', 'Spring', 'Summer'],
                    sharex=False, margin_titles=True,
                    height=3.5, aspect=1.0,
                    #order=['CAT',  'LSTM',  'NHiTS', 'NBEATS', "MLPFPQ"],
                    #palette={"MLVS-PT": "C1", "SPS-UK": "C2"},
                    order=[ 'CAT', 'L-REG',  'MSTL', 'S-Naive'],
                     kind='point',
                   legend='auto', legend_out=False)
    return fig

In [ ]:
test_one=metrics[metrics['Fold']=='test-one']
#test_one=test_one[test_one.Season=='Winter']
test_one['D-type']='Train-test-1'
test_two=metrics[metrics['Fold']=='test-two']
#test_two=test_two[test_two.Season=='Summer']
test_two['D-type']='Train-test-2'
window=metrics[metrics['window']=='expanding']
window['D-type']='Expanding'
data_spiliting=pd.concat([test_one, test_two, window, rolling])

In [ ]:
fig= per_dtype_plot(data_spiliting.reset_index(), metric='mae')
fig.set_axis_labels( 'Model', 'MAE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='lower right', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 10))
#fig.savefig(f'{figure_path}/D-type_MAE.pdf', dpi=480)

In [ ]:
fig= per_dtype_plot(data_spiliting.reset_index(), metric='mape')
fig.set_axis_labels( 'Model', 'MAPE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper right', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 0.6))

In [ ]:
fig= per_dtype_plot(data_spiliting.reset_index(), metric='smape')
fig.set_axis_labels( 'Model', 'SMAPE', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper right', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(0, 0.6))

In [ ]:
fig= per_dtype_plot(data_spiliting.reset_index(), metric='nbias')
fig.set_axis_labels( 'Model', 'NBIAS', fontsize=12)
fig.figure.tight_layout() 
fig.set_titles(row_template="{row_name} Season")
plt.legend(loc='upper left', fontsize=10)
plt.xticks(rotation=90);
fig.set(ylim=(-10, 10))

In [ ]:
data_spiliting.groupby(['D-type', 'Model']).median().sort_values('mae').round(2)[met]

## Computation Metrics

In [ ]:
data=metrics[metrics['window']=='expanding']
data=data[data['Model'].isin([ 'S-Naive','L-REG', 'MSTL', 'RF',  'NHiTS', 'RNN',  'CAT', 'NBEATS'])]

In [ ]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('Model', title=None, sort=[ 'S-Naive','L-REG', 'MSTL', 'RF',  'NHiTS', 'RNN', 'NBEATS', 'CAT']),
    y=alt.Y('Test-time', scale=alt.Scale(domain=[0, 10]), title="Inference time (second)"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20'))
   
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).resolve_scale(x='independent'
).properties(width=100,
                height=120
)

In [ ]:
alt.Chart(data).mark_line(point={
      "filled": False,
      "fill": "white"
    }).encode(
    x=alt.X('Fold', title=None, sort=[ 'S-Naive','L-REG', 'MSTL', 'RF',  'NHiTS', 'RNN', 'NBEATS', 'CAT']),
    y=alt.Y('Train-time', scale=alt.Scale(domain=[0, 10]), title="Train time (minutes)"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau10'))
   
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).properties(width=120,
                height=120
)

In [ ]:
alt.Chart(data).mark_bar().encode(
    x=alt.X('Model', title=None, sort=[ 'S-Naive','L-REG', 'MSTL', 'RF', 'CAT', 'NHiTS', 'RNN', 'NBEATS']),
    y=alt.Y('Train-time', scale=alt.Scale(domain=[0, 10]), title="Train-time (minutes)"),
    color=alt.Color('Model', scale=alt.Scale(scheme='tableau20'))
   
).configure_axis(
    grid=False,
    labelFontSize=12,
    titleFontSize=12
).configure_view(
    strokeOpacity=0
).resolve_scale(x='independent'
).properties(width=100,
                height=120
)

## Compare residual of forecasting models

The residuals of a forecasting model represent the difference between the actual values and the forecasted values Analyzing the residuals, allow to determine how well the model fits the net-load and if there are any patterns or trends that the model is not capturing. For instace if the residuals are consistently positive or negative, this can indicate that the model is systematically overestimating or underestimating the dependent variable. On the other hand, if the model's residuals are small and randomly distributed around zero, this suggests that the model is performing well and accurately predicting the target variable. 

In [ ]:
data=all_data['train_test']
data, real_netload=get_net_load(data)
residual=real_netload.values-data.copy()
best_model=['RNN',  'ARIMA', 'RF']

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 3.5), sharex=True, sharey=True)
for model_name in best_model:
    sns.histplot(residual[model_name], kde=False, ax=ax, fill=False)
ax.legend(best_model)

In [ ]:
from scipy.stats import norm, probplot,  anderson, shapiro
p_values={}
anderson_statics={}
anderson_critical_value={}
fig, ax = plt.subplots(1, 4, figsize=(8, 2), sharex=True, sharey=True)
ax = ax.ravel()
for i, model_name in enumerate(best_model):
    
    sns.histplot(residual[model_name].values, kde=True, ax=ax[i], fill=False)
   
    result = anderson(residual[model_name].values)
    stat, p = shapiro(residual[model_name].values)
    p_values[model_name]=p
    anderson_critical_value[model_name]=result.critical_values[2]
    anderson_statics[model_name]=result.statistic
    # Calculate mean and standard deviation of the residuals
    mean = np.mean(residual[model_name].values)
    std = np.std(residual[model_name].values)
    
        
    if result.statistic < result.critical_values[2]:
        print(f'{model_name} residuals appear to be normally distributed.')
    else:
        print(f'{model_name} residuals do not appear to be normally distributed.')
    # Add a normal distribution curve to the histogram
    x = np.linspace(mean - 3*std, mean + 3*std, 100)
    ax[i].set_title(f"{model_name}")
    ax[i].axvline(mean, color='r', linestyle="--")

    ax[i].set_xlabel("Residual")
    ax[i].set_ylabel("Count")
fig.tight_layout()
fig.savefig(f'{figure_path}/residual_of_models.pdf', dpi=480)

Anderson-Darling test is another widely used test for normality. It is similar to the Shapiro-Wilk test, but it is more sensitive to deviations from normality in the tails of the distribution. The Anderson-Darling test also provides critical values that can be used to evaluate the significance of the test statistic

In [ ]:
import pingouin as pg

for i, model_name in enumerate(metrics.Model.unique()):
    
    data=metrics[metrics['window']=='expanding']
    data_i=data[data['Model'].isin([model_name])]
    

    fig=sns.jointplot(data=data_i, x="res", y="nbias", palette="tab10", height=2.8)
    fig.set_axis_labels('residual', 'NBIAS', fontsize=12)
    fig.figure.tight_layout()
    fig.savefig(f'{figure_path}/{model_name}_joint_nbias.pdf', dpi=480)

In [ ]:

for i, model_name in enumerate(metrics.Model.unique()):
    fig, ax = plt.subplots(1, 1, figsize=(3.2, 2.1), sharex=True, sharey=True)
    data=metrics[metrics['window']=='expanding']
    data_i=data[data['Model'].isin([model_name])]
    ax = pg.qqplot(data_i.res, dist='norm', confidence=False, ax=ax)

    #probplot(residual[model_name].values, dist="norm", plot=ax[i])
    ax.set_title(f"{model_name}")
    ax.set_xlabel("Theoretical Quantiles", fontsize=10)
    ax.set_ylabel("Sample Quantiles",  fontsize=10)
    ax.set_ylim(-3, 5)
    ax.set_xlim(-3, 5)
    fig.tight_layout()
    fig.savefig(f'{figure_path}/{model_name}_probplot_residual.pdf', dpi=480)